## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


**What variable(s) are the target(s) for your model?**
<p>The target variable for the model is "IS_SUCCESSFUL," which represents whether applicants were successful in their funding endeavors (binary: 1 for successful, 0 for not successful).
</p><p>**What variable(s) are the feature(s) for your model?**
<p>The feature variables for the model include the following columns: "APPLICATION_TYPE," "AFFILIATION," "CLASSIFICATION," "USE_CASE," "ORGANIZATION," "STATUS," "INCOME_AMT," "SPECIAL_CONSIDERATIONS," and "ASK_AMT." These features will be used to make predictions about the success of funding applicants.

In [ ]:
#  Let's remove the less relevant 'EIN' and 'NAME' columns.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Uncover and count the number of distinct values
unique_value_counts = application_df.nunique()
unique_value_counts

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [ ]:
# A closer look at the distribution of values in the 'APPLICATION_TYPE' column to guide the binning strategy
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Choose a cutoff value to identify less frequent application types and replace them with 'Other'.
# use the variable name `application_types_to_replace`
cutoff_value = 10
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Replace the chosen application types with 'Other' in the dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Verify to ensure successful binning
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [ ]:
# value counts where count is greater than 1 for the 'CLASSIFICATION' column for specific understanding / to condense output
classification_counts = application_df['CLASSIFICATION'].value_counts()
filtered_classification_counts = classification_counts[classification_counts > 1]
filtered_classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Determine a cutoff value to identify less common classifications and replace them with 'Other'
# Create a list called `classifications_to_replace` to store them
cutoff_value = 100
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [ ]:
# Convert categorical data into a numerical format using one-hot encoding.
# Use pd.get_dummies() to perform one-hot encoding
application_df = pd.get_dummies(application_df)

In [ ]:
# Separate data into features (X) and target (y)
X = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, specifying the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer with 80 nodes and 'relu' activation function
nn.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=X_train.shape[1]))

# Second hidden layer with 30 nodes and 'relu' activation function
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer with a single node and 'sigmoid' activation function for binary classification
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                4320      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6781 (26.49 KB)
Trainable params: 6781 (26.49 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
history = nn.fit(X_train, y_train, epochs=100, batch_size=128, verbose=1)

Epoch 1/100
201/201 [==============================] - 1s 4ms/step - loss: 0.5879 - accuracy: 0.6877
Epoch 2/100
201/201 [==============================] - 1s 2ms/step - loss: 0.5961 - accuracy: 0.6699
Epoch 3/100
201/201 [==============================] - 0s 2ms/step - loss: 0.5900 - accuracy: 0.6792
Epoch 4/100
201/201 [==============================] - 0s 2ms/step - loss: 0.5873 - accuracy: 0.6864
Epoch 5/100
201/201 [==============================] - 0s 2ms/step - loss: 0.5848 - accuracy: 0.6994
Epoch 6/100
201/201 [==============================] - 0s 2ms/step - loss: 0.5985 - accuracy: 0.6755
Epoch 7/100
201/201 [==============================] - 0s 2ms/step - loss: 0.5896 - accuracy: 0.6920
Epoch 8/100
201/201 [==============================] - 0s 2ms/step - loss: 0.7251 - accuracy: 0.6042
Epoch 9/100
201/201 [==============================] - 0s 2ms/step - loss: 0.6456 - accuracy: 0.6168
Epoch 10/100
201/201 [==============================] - 0s 2ms/step - loss: 0.6335 - accura

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define a callback to save the model's weights every five epochs
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"  # Specify the file format and location
checkpoint_callback = ModelCheckpoint(
    checkpoint_path, save_weights_only=True, save_freq=5 * len(X_train), verbose=1
)

# Train the model and use the checkpoint_callback
history = nn.fit(X_train, y_train, epochs=100, batch_size=128, verbose=1, callbacks=[checkpoint_callback])

Epoch 1/100
201/201 [==============================] - 0s 2ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 2/100
201/201 [==============================] - 1s 3ms/step - loss: 0.6913 - accuracy: 0.5318
Epoch 3/100
201/201 [==============================] - 0s 2ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 4/100
201/201 [==============================] - 0s 2ms/step - loss: 0.6913 - accuracy: 0.5318
Epoch 5/100
201/201 [==============================] - 0s 2ms/step - loss: 0.6913 - accuracy: 0.5318
Epoch 6/100
201/201 [==============================] - 0s 2ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 7/100
201/201 [==============================] - 0s 2ms/step - loss: 0.6913 - accuracy: 0.5318
Epoch 8/100
201/201 [==============================] - 0s 2ms/step - loss: 0.6912 - accuracy: 0.5318
Epoch 9/100
201/201 [==============================] - 1s 3ms/step - loss: 0.6913 - accuracy: 0.5318
Epoch 10/100
201/201 [==============================] - 1s 3ms/step - loss: 0.6912 - accura

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 1.3129 - accuracy: 0.6590 - 536ms/epoch - 2ms/step
Loss: 1.3129249811172485, Accuracy: 0.6590087413787842


In [25]:
# Export our model to HDF5 file
miloh_model = " AlphabetSoupCharity.h5"
nn.save(miloh_model)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
